In [1]:
from google import genai
from google.genai.types import EmbedContentConfig

GEMINI_EMBEDDING_MODEL_ID = "text-embedding-large-exp-03-07" 
# Embedding dimension
EMBEDDING_DIMENSION = 3072  # Dimension for text-embedding-005 model

embedding_client = genai.Client(vertexai=True, project="enclaveid", location="us-central1")

# Helper function to generate embeddings using Google Generative AI
async def generate_vertex_embedding(text: str) -> list[float]:
    """Generates embedding for a given text using Google Generative AI."""
    # Use the genai library to get embeddings
    response = embedding_client.models.embed_content(
        model=GEMINI_EMBEDDING_MODEL_ID,
        contents=[text],
        config=EmbedContentConfig(
            task_type="SEMANTIC_SIMILARITY",  # Setting the task type
            output_dimensionality=EMBEDDING_DIMENSION,  # Setting the output dimension
        ),
    )
    
    # Extract the embedding values from the response
    if response and response.embeddings and len(response.embeddings) > 0:
        embedding_values = response.embeddings[0].values
        
        if len(embedding_values) != EMBEDDING_DIMENSION:
            print(f"Warning: Embedding dimension mismatch. Expected {EMBEDDING_DIMENSION}, got {len(embedding_values)}")
            raise ValueError(f"Embedding dimension mismatch. Expected {EMBEDDING_DIMENSION}, got {len(embedding_values)}")
            
        return embedding_values
    else:
        print("Warning: No embedding values returned")
        raise ValueError("No embedding values returned")

In [2]:
import os
import openai

TEACHER_MODEL = "google/gemma-3-27b-it"
JUDGE_MODEL = "google/gemini-2.5-pro-preview-03-25"

openrouter_client = openai.OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY"),
)

def get_completion(model, data) -> str:
    completion = openrouter_client.chat.completions.create(
      extra_body={},
      model=model,
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": data
            },
          ]
        }
      ]
    )
    return completion.choices[0].message.content